In [28]:
!pip install llama-index-llms-openai python-dotenv llama-index gradio > installs.txt

In [6]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [34]:
import gradio as gr
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select
from IPython.display import Markdown, display
from llama_index.core.query_engine import NLSQLTableQueryEngine

# Define the function to execute the query
def execute_query(tables, query_str):
    db_file_path = "company.db"
    engine = create_engine(f"sqlite:///{db_file_path}")
    llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
    sql_database = SQLDatabase(engine, include_tables=tables)
    query_engine = NLSQLTableQueryEngine(
        sql_database=sql_database, tables=tables, llm=llm,
    )
    response = query_engine.query(query_str)
    response_text = str(response)
    sql_query = response.metadata['sql_query']
    return response_text, sql_query

iface = gr.Interface(
    fn=execute_query,
    inputs=[
        gr.CheckboxGroup(["employees", "salaries", "projects", "departments"], label="Select tables"),
        gr.Textbox(lines=2, placeholder="Enter your query string"),
    ],
    outputs=[
        gr.Textbox(label="Response"),
        gr.Textbox(label="SQL Query"),
    ],
    title="Natural Language to SQL Query Engine",
    description="Enter the table names, query string, and your SQLAlchemy engine string to execute the query.",
)

iface.launch()


Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
